# Tensorflow 실습 4 : 딥러닝 구현 실습하기

### 본 실습의 목적
  - 딥러닝 template을 응용해서 처음부터 끝까지 코드를 작성해보기
  - 수업에서 배우지 않은 부분도 documentation을 검색하여 활용할 수 있는 능력 기르기

- 참고: GPU를 이용하면 학습 속도가 빠릅니다.

### 데이터셋

In [ ]:
import os
import tensorflow as tf
from time import localtime, time
import numpy as np
import matplotlib.pyplot as plt

- 사용할 데이터셋은 "CIFAR-10" 데이터셋
- 총 10개의 class
- 본 실습에서는 각 이미지를 알맞은 class로 분류하는 딥러닝 모델을 학습하기 위해 사용
<table>
  <tr><td>
    <img src="https://user-images.githubusercontent.com/15958325/63308580-41b7fe80-c32e-11e9-827f-98052675c0ea.png">
  </td></tr>
  <tr><td align="center">
    <b>그림 1.</b> CIFAR-10 샘플 이미지 <br/>&nbsp;
  </td></tr>
</table>


In [ ]:
# cifar10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# cifar10 dataset
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

# 데이터를 0.8:0.2 비율로 traing, validation 데이터로 분리
ratio = 0.8
x_train, x_valid = x_train[:int(len(x_train)*ratio)], x_train[int(len(x_train)*ratio):]
y_train, y_valid = y_train[:int(len(y_train)*ratio)], y_train[int(len(y_train)*ratio):]

print('이미지 shpae: ', x_train[1].shape)
print('정답 label: ', y_train[1])
print('training 데이터 개수: ', len(x_train))
print('validation 데이터 개수: ', len(x_valid))
print('test 데이터 개수: ', len(x_test))

# 이미지 예시
plt.imshow(x_train[1])
plt.title(class_names[y_train[1][0]])
plt.show()

## 문제 1 : tf.data.Dataset 이용하기


- 위에서 load 된 데이터를 이용하여, tf.data.Dataset 만들기
  - 조건 1: training, validation, test 데이터에 대해 각각 만들기
  - 조건 2: 모든 데이터셋에 대해, batch size는 64로 설정하기
  - 조건 3: training 데이터셋은 shuffle 적용하기 (buffer size는 40000)

In [ ]:
# 여기에 문제 1에 대한 코드를 작성하면 됩니다.

## 문제 2 : 딥러닝 모델 선언하기
- 조건: 딥러닝 모델의 각 layer는 1번부터 9번까지 순서대로 구성
1. input을 하나의 axis만 가지도록 flatten
2. dense layer, hidden node 256, ReLU activation
3. batch normalization
4. dense layer, hidden node 128, activation 없음
5. LeakyReLU activation function (alpha=0.2) 
(Hint: tf.keras.layers에서 검색하기)
6. layer normalization (Hint: 검색하기)
7. dense layer, hidden node 64, ReLU activation
8. dropout, 확률 0.2
9. dense layer, hidden node 10, softmax activation
- 딥러닝 모델을 만드는 두가지 방식(tf.keras.Model, tf.keras.models.Sequential) 중 선택하여 만들기

In [ ]:
# 여기에 문제 2에 대한 코드를 작성하면 됩니다.

## 문제 3 : 옵티마이저, 손실 함수, 평가 metric 선택하기
- 조건1: 옵티마이저는 adam을 사용, learning rate는 0.001
- 조건2: 손실함수는 multi-class classification에 적합한 함수를 선택
- 조건3: 평가 metric은 training, valdation에 대해 각각 loss 및 accuracy

In [ ]:
# 여기에 문제 3에 대한 코드를 작성하면 됩니다.

## step 함수 정의: 기존 template과 동일함

- mini batch data를 가지고 연산하는 과정을 이해하기

In [ ]:
# @tf.function은 Autograph (일반적으로 속도 향상)
@tf.function
def train_step(images, labels, model, loss_metric, accuracy_metric):
    with tf.GradientTape() as tape:
        # model에 input을 넣어주면, call 함수가 실행 
        predictions = model(images, training=True)
        # loss_object를 이용하여 loss를 계산함
        loss = loss_object(labels, predictions)
    # 모델의 학습 가능한 파라미터들에 대한 gradient 값을 계산
    gradients = tape.gradient(loss, model.trainable_variables)
    # 모델의 각 파라미터에 대해, gradients를 적용
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # loss와 accuracy를 metric에 저장
    loss_metric(loss)
    accuracy_metric(labels, predictions)

def validation_and_test_step(images, labels, model, loss_metric, accuracy_metric):
    predictions = model(images, training=False)
    loss = loss_object(labels, predictions)
    loss_metric(loss)
    accuracy_metric(labels, predictions)

## tensorboard 준비


In [ ]:
# 학습과정 summary 저장 및 시각화를 위한 tensorboard log file
tb_path = os.path.join('tensorboard','tb-%02d%02d-%02d%02d' % tuple(localtime(time()))[1:5])
summary_writer = tf.summary.create_file_writer(tb_path)

## 문제 4 : 딥러닝 모델 학습 및 저장하기
- 조건1: 20 epoch 동안 학습하기
- 조건2: 매번 epoch이 끝날 때마다, 각각 다른 파일로 학습된 모델을 저장하기 (참고: 몇 번째 epoch의 모델인지를 기록하는 것이 좋음)
- 조건3: 매번 epoch 마다 학습 및 validation을 수행하고, 평가 metric 값들을 출력하기
- 조건4: tensorboard를 이용하여, 매번 epoch이 끝날 때마다 두 가지 loss값(train loss, valid loss)를 기록하기

In [ ]:
# 여기에 문제 4에 대한 코드를 작성하면 됩니다.

## tensorboard로 시각화


In [ ]:
# Colab 환경에서 tensorboard를 사용하는 방법
%load_ext tensorboard
%tensorboard --logdir tensorboard/

## 문제 5: Best 모델 불러오기 및 테스트 성능 평가

- 조건1: best 모델은 전체 epoch 중에 validation loss가 가장 낮은 epoch의 모델로 선택
- 조건2: 테스트 데이터에 대해, loss 값과 accuracy 값을 출력하기

In [ ]:
# 여기에 문제 5에 대한 코드를 작성하면 됩니다

## 문제 6: 테스트 이미지에 대한 예측값 출력하기
- 조건1: 테스트 이미지(x_test) 중 0번 ~ 9번까지 총 10개의 이미지에 대해 예측 label 출력하기
- 조건2: 테스트 이미지(x_test) 중 0번 ~ 9번까지 총 10개의 이미지를 시각화하여, 예측값과 이미지 비교하기 (for문 이용)

In [ ]:
# 여기에 문제 6에 대한 코드를 작성하면 됩니다